In [1]:
import numpy as np
import pandas as pd
import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, PReLU
import tensorflow as tf
import gc
tf.__version__

'2.8.0'

In [2]:
train = pd.read_csv('../../input/Allstate/train.csv.zip')
test = pd.read_csv('../../input/Allstate/test.csv.zip')
submission = pd.read_csv('../../input/Allstate/sample_submission.csv.zip')

In [3]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [4]:
train.shape

(188318, 132)

In [5]:
test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [6]:
test.shape

(125546, 131)

In [9]:
np.array(list(test.columns))

array(['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14',
       'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21',
       'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28',
       'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35',
       'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42',
       'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49',
       'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56',
       'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63',
       'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70',
       'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77',
       'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84',
       'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91',
       'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98',
       'cat99', 'cat100

In [10]:
np.random.seed(123)
index = list(train.index)
print(index[0:10])
np.random.shuffle(index)
print(index[0:10])
train = train.iloc[index]
'train = train.iloc[np.random.permutation(len(train))]'

## set test loss to NaN
test['loss'] = np.nan

## response and IDs
## We are log-transforming the target

y = np.log(train['loss'].values+200)
id_train = train['id'].values
id_test = test['id'].values

## stack train test
ntrain = train.shape[0]
tr_te = pd.concat((train, test), axis = 0)

## Preprocessing and transforming to sparse data
sparse_data = []

f_cat = [f for f in tr_te.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(tr_te[f].astype('category'))
    tmp = csr_matrix(dummy)
    sparse_data.append(tmp)

f_num = [f for f in tr_te.columns if 'cont' in f]
scaler = StandardScaler()
tmp = csr_matrix(scaler.fit_transform(tr_te[f_num]))
sparse_data.append(tmp)

del(tr_te, train, test)

## sparse train and test data
xtr_te = hstack(sparse_data, format = 'csr')
xtrain = xtr_te[:ntrain, :]
xtest = xtr_te[ntrain:, :]

print('Dim train', xtrain.shape)
print('Dim test', xtest.shape)

del(xtr_te, sparse_data, tmp)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[33898, 43407, 3458, 100139, 60086, 117290, 89806, 7751, 164059, 143297]
Dim train (188318, 1190)
Dim test (125546, 1190)


In [11]:
y

array([7.56987608, 7.84829365, 7.41602364, ..., 7.33236921, 9.17228019,
       6.37051723])

In [13]:
# Separate features and targets
X = xtrain.toarray()
Y = y

# Set the input shape
input_shape = (xtrain.shape[1],)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Dense(16, input_shape=input_shape, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
del xtrain
gc.collect()
gc.collect()

Feature shape: (1190,)


2022-06-06 07:24:02.546268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 07:24:02.555097: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 07:24:02.555554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 07:24:02.556647: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

0

In [14]:
X

array([[ 0.        ,  1.        ,  1.        , ..., -0.95914517,
        -0.61232072, -1.26329393],
       [ 1.        ,  0.        ,  1.        , ...,  0.39395194,
        -0.71240011, -1.1963163 ],
       [ 1.        ,  0.        ,  1.        , ...,  0.91652738,
         0.94750019, -0.90385702],
       ...,
       [ 0.        ,  1.        ,  1.        , ..., -0.83567771,
        -0.01839175,  1.55127219],
       [ 1.        ,  0.        ,  0.        , ..., -0.23903515,
        -0.80958157, -0.56602101],
       [ 0.        ,  1.        ,  1.        , ...,  0.39395194,
         1.53407398, -0.95125139]])

In [18]:
## neural net
def nn_model():
    model = Sequential()
    
    model.add(Dense(400, input_dim = X.shape[1], kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
        
    model.add(Dense(200, kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(50, kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, kernel_initializer = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adadelta')
    return(model)

In [22]:
Y

array([7.56987608, 7.84829365, 7.41602364, ..., 7.33236921, 9.17228019,
       6.37051723])

In [26]:
%%time

test_preds_total = 0
N_SPLITS = 5

kf = KFold(n_splits=N_SPLITS, random_state=137, shuffle=True)
for train_index, val_index in kf.split(X):
    print("TRAIN:", train_index, "VAL:", val_index)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = nn_model()
    model.fit(X_train, y_train, epochs=600, batch_size=1024, verbose=0)
    
    val_preds = np.exp(model.predict(X_val))-200
    error = mean_absolute_error(np.exp(y_val)-200, val_preds)
    
    print(f'Fold mae: {error}')
    
    model.compile(loss='mae', optimizer='adam', metrics=['mae'])
    model.fit(X_train, y_train, epochs=20, batch_size=100, verbose=0)
    
    val_preds = np.exp(model.predict(X_val))-200
    error = mean_absolute_error(np.exp(y_val)-200, val_preds)
    
    print(f'Fold mae: {error}')
    
    model.compile(loss='mae', optimizer='adam', metrics=['mae'])
    model.fit(X_train, y_train, epochs=20, batch_size=50, verbose=0)
    
    val_preds = np.exp(model.predict(X_val))-200
    error = mean_absolute_error(np.exp(y_val)-200, val_preds)
    
    print(f'Fold mae: {error}')
    
    test_preds = np.exp(model.predict(xtest))-200
    test_preds_total += test_preds/N_SPLITS


TRAIN: [     1      2      4 ... 188313 188314 188315] VAL: [     0      3      6 ... 188309 188316 188317]
Fold mae: 1426.1004693510822
Fold mae: 1151.7130172695136
Fold mae: 1147.834260356096
TRAIN: [     0      2      3 ... 188315 188316 188317] VAL: [     1      5      8 ... 188300 188305 188306]
Fold mae: 1482.6282695513896
Fold mae: 1149.7667657346246
Fold mae: 1154.610037790005
TRAIN: [     0      1      2 ... 188314 188316 188317] VAL: [    16     17     30 ... 188310 188311 188315]
Fold mae: 1436.7284508900086
Fold mae: 1160.4965557379712
Fold mae: 1154.5343910471383
TRAIN: [     0      1      3 ... 188315 188316 188317] VAL: [     2      7     10 ... 188299 188307 188313]
Fold mae: 1454.026424503341
Fold mae: 1132.8270284451994
Fold mae: 1138.7273166313523
TRAIN: [     0      1      2 ... 188315 188316 188317] VAL: [     4     13     25 ... 188308 188312 188314]
Fold mae: 1581.4770286744435
Fold mae: 1156.7994244260467
Fold mae: 1139.7938525847621
CPU times: user 1h 55min 14s

In [27]:
test_preds_total

array([[1570.043  ],
       [1765.6118 ],
       [9318.161  ],
       ...,
       [2371.4565 ],
       [ 969.34357],
       [3233.838  ]], dtype=float32)

In [28]:
val_preds

array([[9739.92  ],
       [1294.9211],
       [3418.0183],
       ...,
       [3772.5396],
       [2409.1924],
       [4687.072 ]], dtype=float32)

In [29]:
submission['loss'] = test_preds_total
submission.to_csv('../submissions/sub_simple_mlp_5_fold_0.csv', index=False)
submission.head()

,id,loss
0,4,1570.042969
1,6,1765.611816
2,9,9318.161133
3,12,7524.106934
4,15,826.004578


This submission scores 1115.43903 on the public leaderboard, and 1127.08074 on the private one.
